# Initial Data View

In [7]:
from sklearn.datasets import load_wine
from sklearn.linear_model import LogisticRegression
import pandas as pd

# Load dataset
wine = load_wine()
df = pd.DataFrame(wine.data, columns=wine.feature_names)

df['target'] = wine.target
df = df[df['target'] < 2]


In [8]:
print(df)

     alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols   
0      14.23        1.71  2.43               15.6      127.0           2.80  \
1      13.20        1.78  2.14               11.2      100.0           2.65   
2      13.16        2.36  2.67               18.6      101.0           2.80   
3      14.37        1.95  2.50               16.8      113.0           3.85   
4      13.24        2.59  2.87               21.0      118.0           2.80   
..       ...         ...   ...                ...        ...            ...   
125    12.07        2.16  2.17               21.0       85.0           2.60   
126    12.43        1.53  2.29               21.5       86.0           2.74   
127    11.79        2.13  2.78               28.5       92.0           2.13   
128    12.37        1.63  2.30               24.5       88.0           2.22   
129    12.04        4.30  2.38               22.0       80.0           2.10   

     flavanoids  nonflavanoid_phenols  proanthocyan

In [122]:
from sklearn.model_selection import train_test_split
# Prepare features and labels
X = df.drop(columns=['target'])
y = df['target']


# Split the data into train and test sets (80% train, 20% test) with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Train logistic regression model
model = LogisticRegression(penalty='l2', solver='saga')
model.fit(X_train, y_train)

# Evaluate model
accuracy = model.score(X_test, y_test)

print("Accuracy:", accuracy)

Accuracy: 0.9615384615384616


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [163]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.linear_model import LinearRegression

# ---------------------------
# 1. Load and preprocess data
# ---------------------------
wine = load_wine()
df = pd.DataFrame(wine.data, columns=wine.feature_names)

df['target'] = wine.target
# Restrict to classes 0 and 1
df = df[df['target'] < 2]

X = df[wine.feature_names].values   # Shape: (N, d)
y = df['target'].values            # Shape: (N,)

# ---------------------------
# 2. Normalize data so that each feature is in [-1, 1]
#    X_norm[j] = 2 * (X[j] - min(X[j])) / (max(X[j]) - min(X[j])) - 1
# ---------------------------
X_min = X.min(axis=0)
X_max = X.max(axis=0)
X_range = X_max - X_min

# Safeguard against division by zero: if a feature is constant, keep it at 0
X_range[X_range == 0] = 1e-9

X_norm = 2.0 * (X - X_min) / X_range - 1.0

# ---------------------------
# 3. Initialize weights using a simple line of best fit (LinearRegression)
#    We'll do a linear regression on X_norm -> y, then store intercept + coefs.
# ---------------------------
linreg = LinearRegression()
linreg.fit(X_norm, y)

# Extract coefficients and intercept
# w_init will include intercept as w[0], followed by coefficients
w_init = np.concatenate(([linreg.intercept_], linreg.coef_))

# Augment X with a column of ones so that we can treat the intercept
# as just another weight dimension.
# New shape of X_aug: (N, d+1)
X_aug = np.hstack([np.ones((X_norm.shape[0], 1)), X_norm])

# ---------------------------
# 4. Define helper functions
# ---------------------------
def sigmoid(z):
    """
    The logistic sigmoid function.
    """
    return 1 / (1 + np.exp(-z))

def logistic_loss(X_aug, y, w):
    """
    Computes the average negative log-likelihood (logistic loss).
    """
    eps = 1e-10  # For numerical stability
    predictions = sigmoid(X_aug @ w)  # Shape: (N,)
    N = X_aug.shape[0]
    loss_val = -1.0 / N * np.sum(
        y * np.log(predictions + eps) + (1 - y) * np.log(1 - predictions + eps)
    )
    return loss_val

def compute_gradient(X_aug, y, w):
    """
    Computes the gradient of the negative log-likelihood (logistic loss).
    X_aug: (N, d+1) with the first column all ones (for intercept).
    y:     (N,) binary labels in {0, 1}.
    w:     (d+1,) current weight vector (including intercept).
    
    Returns:
       grad: (d+1,) gradient of logistic loss w.r.t. w
    """
    N = X_aug.shape[0]
    predictions = sigmoid(X_aug @ w)      # Shape: (N,)
    grad = (1.0 / N) * (X_aug.T @ (predictions - y))
    return grad

def get_largest_gradient_dimension(X_aug, y, w):
    """
    1. Compute the gradient for the current parameter w.
    2. Find the index of the dimension with largest absolute partial derivative.
    
    Returns:
       grad: the gradient (d+1,)
       idx:  index of the largest (by absolute value) partial derivative
    """
    grad = compute_gradient(X_aug, y, w)
    idx = np.argmax(np.abs(grad))
    return grad, idx

def coordinate_descent_update(X_aug, y, w, idx, step_size, grad, num_steps=10):
    """
    Performs repeated updates on a single coordinate (dimension) 'idx' with a fixed
    step size. We re-compute the gradient each step because as we update w[idx],
    the partial derivative changes.
    
    After each update, print the current loss.
    
    X_aug: (N, d+1) data
    y:     (N,)     labels
    w:     (d+1,)   weights (updated in place)
    idx:   int      which dimension to update
    step_size: float
    num_steps: int   how many small updates to perform in that coordinate
    """
    current_loss = logistic_loss(X_aug, y, w)
    print(f"[Coordinate {idx}, Pre-Update Loss = {current_loss:.4f}")
    for step in range(num_steps):
        # Re-compute gradient
        
        
        # Perform coordinate descent update in the negative gradient direction
        w[idx] -= step_size * grad[idx]
        
        # Print loss after each update
        new_loss = logistic_loss(X_aug, y, w)

        if new_loss > current_loss:
            w[idx] += grad[idx] * step_size
            print(f"Stopped Early after {step} steps")
            break
        else:
            current_loss = new_loss
        
    print(f"[Coordinate {idx}, Post-Update Loss = {current_loss:.4f}")

# ---------------------------
# 5. Run the procedure
# ---------------------------

# Copy initial weights (so we don't overwrite w_init)
w = w_init.copy()

print("Initial loss:", logistic_loss(X_aug, y, w))

# Choose a fixed step size
step_size = 0.01

# Number of outer iterations over coordinates
num_outer_iters = 100

for iteration in range(num_outer_iters):
    print(f"\n--- Outer Iteration {iteration+1}/{num_outer_iters} ---")
    # 1. Find the coordinate with largest partial derivative
    grad, idx = get_largest_gradient_dimension(X_aug, y, w)
    
    # 2. Update that coordinate repeatedly
    coordinate_descent_update(X_aug, y, w, idx, step_size, grad, num_steps=50)

print("Final loss:", logistic_loss(X_aug, y, w))


Initial loss: 0.5158783355150558

--- Outer Iteration 1/100 ---
[Coordinate 1, Pre-Update Loss = 0.5159
[Coordinate 1, Post-Update Loss = 0.5051

--- Outer Iteration 2/100 ---
[Coordinate 1, Pre-Update Loss = 0.5051
[Coordinate 1, Post-Update Loss = 0.4948

--- Outer Iteration 3/100 ---
[Coordinate 1, Pre-Update Loss = 0.4948
[Coordinate 1, Post-Update Loss = 0.4850

--- Outer Iteration 4/100 ---
[Coordinate 1, Pre-Update Loss = 0.4850
[Coordinate 1, Post-Update Loss = 0.4757

--- Outer Iteration 5/100 ---
[Coordinate 1, Pre-Update Loss = 0.4757
[Coordinate 1, Post-Update Loss = 0.4667

--- Outer Iteration 6/100 ---
[Coordinate 1, Pre-Update Loss = 0.4667
[Coordinate 1, Post-Update Loss = 0.4582

--- Outer Iteration 7/100 ---
[Coordinate 1, Pre-Update Loss = 0.4582
[Coordinate 1, Post-Update Loss = 0.4501

--- Outer Iteration 8/100 ---
[Coordinate 1, Pre-Update Loss = 0.4501
[Coordinate 1, Post-Update Loss = 0.4424

--- Outer Iteration 9/100 ---
[Coordinate 1, Pre-Update Loss = 0.4424


In [169]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.linear_model import LinearRegression

# ---------------------------
# 1. Load and preprocess data
# ---------------------------
wine = load_wine()
df = pd.DataFrame(wine.data, columns=wine.feature_names)

df['target'] = wine.target
# Restrict to classes 0 and 1
df = df[df['target'] < 2]

X = df[wine.feature_names].values   # Shape: (N, d)
y = df['target'].values            # Shape: (N,)

# ---------------------------
# 2. Normalize each feature to be in [-1, 1]
#    X_norm[j] = 2 * (X[j] - min(X[j])) / (max(X[j]) - min(X[j])) - 1
# ---------------------------
X_min = X.min(axis=0)
X_max = X.max(axis=0)
X_range = X_max - X_min

# Safeguard: If a feature is constant, replace zero range by small value
X_range[X_range == 0] = 1e-9

X_norm = 2.0 * (X - X_min) / X_range - 1.0

# ---------------------------
# 3. Initialize weights using a simple line of best fit (LinearRegression)
#    We'll fit linear regression on X_norm -> y, then extract intercept + coefs.
# ---------------------------
linreg = LinearRegression()
linreg.fit(X_norm, y)

# w_init will include intercept as w[0], followed by coefficients
w_init = np.concatenate(([linreg.intercept_], linreg.coef_))

# Augment X with a column of ones (for the intercept),
# so X_aug has shape (N, d+1).
X_aug = np.hstack([np.ones((X_norm.shape[0], 1)), X_norm])

# ---------------------------
# 4. Define helper functions
# ---------------------------
def sigmoid(z):
    """
    Logistic sigmoid function.
    """
    return 1 / (1 + np.exp(-z))

def logistic_loss(X_aug, y, w):
    """
    Computes the average negative log-likelihood (logistic loss).
    """
    eps = 1e-10  # For numerical stability
    predictions = sigmoid(X_aug @ w)  # Shape: (N,)
    N = X_aug.shape[0]
    loss_val = -1.0 / N * np.sum(
        y * np.log(predictions + eps) + (1 - y) * np.log(1 - predictions + eps)
    )
    return loss_val

def find_best_coordinate_and_direction(X_aug, y, w, step_size):
    """
    For each coordinate i, we try moving w[i] + step_size and w[i] - step_size.
    We see how much each move decreases the logistic loss.

    We pick the coordinate and direction (+/-) that yields the greatest
    DECREASE in the loss.

    Returns:
      best_coord:    The index (0..d) that yields the largest improvement
      best_direction: +1 or -1, indicating which direction is best
      best_improvement: The amount of loss decrease achieved by that coordinate/direction
    """
    current_loss = logistic_loss(X_aug, y, w)
    best_coord = 0
    best_direction = 0
    best_improvement = 0.0

    d_plus_one = len(w)

    for i in range(d_plus_one):
        # Try w[i] + step_size
        w_temp_plus = w.copy()
        w_temp_plus[i] += step_size
        loss_plus = logistic_loss(X_aug, y, w_temp_plus)
        improvement_plus = current_loss - loss_plus  # positive if the loss went down

        # Try w[i] - step_size
        w_temp_minus = w.copy()
        w_temp_minus[i] -= step_size
        loss_minus = logistic_loss(X_aug, y, w_temp_minus)
        improvement_minus = current_loss - loss_minus

        # Determine which direction yields the larger improvement for this coordinate
        if improvement_plus > improvement_minus:
            improvement_for_i = improvement_plus
            direction_for_i = +1
        else:
            improvement_for_i = improvement_minus
            direction_for_i = -1

        # Check if this is better than our current best
        if improvement_for_i > best_improvement:
            best_improvement = improvement_for_i
            best_coord = i
            best_direction = direction_for_i

    return best_coord, best_direction, best_improvement

def coordinate_descent_update(X_aug, y, w, coord, direction, step_size, num_steps=5):
    """
    Once we've chosen which coordinate and direction yields the best
    immediate improvement, we can do multiple steps in that direction.

    For each step, we update w[coord] by 'direction * step_size'
    and print the new loss.
    """
    curr_loss = logistic_loss(X_aug, y, w)
    print(f"Pre-Update Loss = {curr_loss:.4f}")
    
    for step in range(num_steps):
        
        w[coord] += direction * step_size
        new_loss = logistic_loss(X_aug, y, w)
        if new_loss > curr_loss:
            w[coord] -= direction * step_size
            print(f"Stopped Early after {step} steps")
            break
        else:
            curr_loss = new_loss
    print(f"Post-Update Loss = {logistic_loss(X_aug, y, w):.4f}")
        

# ---------------------------
# 5. Run the procedure
# ---------------------------
w = w_init.copy()
initial_loss = logistic_loss(X_aug, y, w)

print(f"Initial Loss: {initial_loss:.4f}")

num_outer_iters = 100  # Number of times we scan and pick the best coordinate
step_size = 0.01      # Fixed step size
num_steps = 50        # How many times we move in that chosen direction once found

for iteration in range(num_outer_iters):
    print(f"\n--- Outer Iteration {iteration+1}/{num_outer_iters} ---")
    best_coord, best_direction, best_improvement = find_best_coordinate_and_direction(X_aug, y, w, step_size)

    if best_improvement <= 0:
        # No improvement found by any coordinate update (could happen if we're at a local minimum or step is too large)
        print("No further improvement found. Stopping early.")
        break

    # Otherwise, do multiple steps in that chosen coordinate and direction
    print(f"Best coordinate: {best_coord}, direction: {best_direction}, improvement: {best_improvement:.4f}")
    coordinate_descent_update(X_aug, y, w, best_coord, best_direction, step_size, num_steps=num_steps)

# Final result
final_loss = logistic_loss(X_aug, y, w)
print("\nFinal weights (including intercept):", w)
print(f"Final Loss: {final_loss:.4f}")


Initial Loss: 0.5159

--- Outer Iteration 1/100 ---
Best coordinate: 1, direction: -1, improvement: 0.0015
Pre-Update Loss = 0.5159
Post-Update Loss = 0.4479

--- Outer Iteration 2/100 ---
Best coordinate: 1, direction: -1, improvement: 0.0012
Pre-Update Loss = 0.4479
Post-Update Loss = 0.3910

--- Outer Iteration 3/100 ---
Best coordinate: 13, direction: -1, improvement: 0.0011
Pre-Update Loss = 0.3910
Post-Update Loss = 0.3439

--- Outer Iteration 4/100 ---
Best coordinate: 1, direction: -1, improvement: 0.0009
Pre-Update Loss = 0.3439
Post-Update Loss = 0.3028

--- Outer Iteration 5/100 ---
Best coordinate: 1, direction: -1, improvement: 0.0007
Pre-Update Loss = 0.3028
Post-Update Loss = 0.2693

--- Outer Iteration 6/100 ---
Best coordinate: 0, direction: -1, improvement: 0.0006
Pre-Update Loss = 0.2693
Stopped Early after 42 steps
Post-Update Loss = 0.2558

--- Outer Iteration 7/100 ---
Best coordinate: 13, direction: -1, improvement: 0.0008
Pre-Update Loss = 0.2558
Post-Update Los

In [168]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.linear_model import LinearRegression

# ---------------------------
# 1. Load and preprocess data
# ---------------------------
wine = load_wine()
df = pd.DataFrame(wine.data, columns=wine.feature_names)

df['target'] = wine.target
# Restrict to classes 0 and 1
df = df[df['target'] < 2]

X = df[wine.feature_names].values   # Shape: (N, d)
y = df['target'].values            # Shape: (N,)

# ---------------------------
# 2. Normalize each feature to be in [-1, 1]
#    X_norm[j] = 2 * (X[j] - min(X[j])) / (max(X[j]) - min(X[j])) - 1
# ---------------------------
X_min = X.min(axis=0)
X_max = X.max(axis=0)
X_range = X_max - X_min

# Safeguard: If a feature is constant, replace zero range by small value
X_range[X_range == 0] = 1e-9

X_norm = 2.0 * (X - X_min) / X_range - 1.0

# ---------------------------
# 3. Initialize weights using a simple line of best fit (LinearRegression)
#    We'll fit linear regression on X_norm -> y, then extract intercept + coefs.
# ---------------------------
linreg = LinearRegression()
linreg.fit(X_norm, y)

# w_init will include intercept as w[0], followed by coefficients
w_init = np.concatenate(([linreg.intercept_], linreg.coef_))

# Augment X with a column of ones (for the intercept),
# so X_aug has shape (N, d+1).
X_aug = np.hstack([np.ones((X_norm.shape[0], 1)), X_norm])

# ---------------------------
# 4. Define helper functions
# ---------------------------
def sigmoid(z):
    """
    Logistic sigmoid function.
    """
    return 1 / (1 + np.exp(-z))

def logistic_loss(X_aug, y, w):
    """
    Computes the average negative log-likelihood (logistic loss).
    """
    eps = 1e-10  # For numerical stability in log
    predictions = sigmoid(X_aug @ w)  # Shape: (N,)
    N = X_aug.shape[0]
    loss_val = -1.0 / N * np.sum(
        y * np.log(predictions + eps) + (1 - y) * np.log(1 - predictions + eps)
    )
    return loss_val

def pick_random_coordinate_and_direction(X_aug, y, w, step_size):
    """
    1) Pick a single coordinate at random.
    2) For that coordinate, check w[i] + step_size and w[i] - step_size.
    3) Pick the direction (+1 or -1) that yields the largest decrease in the logistic loss.
       If neither direction improves the loss, we return zero improvement.

    Returns:
      coord: int (the randomly selected coordinate index)
      best_direction: +1 or -1 (the direction that yields the best immediate improvement)
      best_improvement: how much that update reduces the loss (>= 0)
    """
    current_loss = logistic_loss(X_aug, y, w)
    coord = np.random.randint(0, len(w))  # pick a random index

    # Try a +step_size move
    w_temp_plus = w.copy()
    w_temp_plus[coord] += step_size
    loss_plus = logistic_loss(X_aug, y, w_temp_plus)
    improvement_plus = current_loss - loss_plus

    # Try a -step_size move
    w_temp_minus = w.copy()
    w_temp_minus[coord] -= step_size
    loss_minus = logistic_loss(X_aug, y, w_temp_minus)
    improvement_minus = current_loss - loss_minus

    # Compare which direction is better for that coordinate
    if improvement_plus > improvement_minus:
        best_direction = +1
        best_improvement = improvement_plus
    else:
        best_direction = -1
        best_improvement = improvement_minus

    # If neither direction improves the loss, best_improvement will be <= 0
    return coord, best_direction, best_improvement

def coordinate_descent_update(X_aug, y, w, coord, direction, step_size, num_steps=5):
    """
    Once we've picked the coordinate and direction, 
    perform multiple updates along that direction (if beneficial).

    For each step:
      - Update w[coord] by (direction * step_size).
      - Stop early if the update does not decrease the loss.

    Print the loss before and after these updates.
    """
    curr_loss = logistic_loss(X_aug, y, w)
    print(f"   Pre-Update Loss = {curr_loss:.4f}")
    
    for step in range(num_steps):
        # Try the update
        w[coord] += direction * step_size
        new_loss = logistic_loss(X_aug, y, w)

        if new_loss > curr_loss:
            # If the new loss is worse, revert the change and stop
            w[coord] -= direction * step_size
            print(f"   Stopped Early after {step} steps (no improvement).")
            break
        else:
            # We got an improvement
            curr_loss = new_loss
    
    print(f"   Post-Update Loss = {logistic_loss(X_aug, y, w):.4f}")

# ---------------------------
# 5. Run the procedure
# ---------------------------
w = w_init.copy()
initial_loss = logistic_loss(X_aug, y, w)

print(f"Initial Loss: {initial_loss:.4f}")

num_outer_iters = 100  # How many times we pick a random coordinate and attempt an update
step_size = 0.01       # Fixed step size
num_steps = 50         # Max updates per chosen coordinate

for iteration in range(num_outer_iters):
    print(f"\n--- Outer Iteration {iteration+1}/{num_outer_iters} ---")
    
    # Pick one coordinate at random, find the best direction among {+1, -1}
    coord, direction, improvement = pick_random_coordinate_and_direction(X_aug, y, w, step_size)

    if improvement <= 0:
        # No improvement found from either +step or -step on this random coordinate
        print(f"Coordinate {coord} yields no improvement in either direction. No update performed.")
        # We continue to the next iteration (we do NOT stop training entirely, 
        # because another random coordinate might still help).
        continue

    # Otherwise, descend along that coordinate/direction
    print(f"Random coordinate: {coord}, direction: {direction}, improvement: {improvement:.6f}")
    coordinate_descent_update(X_aug, y, w, coord, direction, step_size, num_steps=num_steps)

# Final result
final_loss = logistic_loss(X_aug, y, w)
print("\nFinal weights (including intercept):", w)
print(f"Final Loss: {final_loss:.4f}")


Initial Loss: 0.5159

--- Outer Iteration 1/100 ---
Random coordinate: 7, direction: -1, improvement: 0.000641
   Pre-Update Loss = 0.5159
   Post-Update Loss = 0.4870

--- Outer Iteration 2/100 ---
Random coordinate: 8, direction: 1, improvement: 0.000713
   Pre-Update Loss = 0.4870
   Post-Update Loss = 0.4573

--- Outer Iteration 3/100 ---
Random coordinate: 4, direction: 1, improvement: 0.000652
   Pre-Update Loss = 0.4573
   Post-Update Loss = 0.4286

--- Outer Iteration 4/100 ---
Random coordinate: 5, direction: -1, improvement: 0.000262
   Pre-Update Loss = 0.4286
   Post-Update Loss = 0.4210

--- Outer Iteration 5/100 ---
Random coordinate: 13, direction: -1, improvement: 0.001111
   Pre-Update Loss = 0.4210
   Post-Update Loss = 0.3720

--- Outer Iteration 6/100 ---
Random coordinate: 11, direction: 1, improvement: 0.000261
   Pre-Update Loss = 0.3720
   Post-Update Loss = 0.3627

--- Outer Iteration 7/100 ---
Random coordinate: 5, direction: -1, improvement: 0.000022
   Pre-U